In [9]:
import os
from os.path import join, dirname, abspath
from dotenv import load_dotenv

# Get the directory where the current script is located
script_directory = dirname(abspath("/"))

# Construct the path to the .env file relative to the script directory
dotenv_path = join(script_directory, '.env')
load_dotenv(dotenv_path)

UNSPLASH_ACCESS_KEY = os.environ.get("UNSPLASH_ACCESS_KEY")

if UNSPLASH_ACCESS_KEY:
    print("UNSPLASH_ACCESS_KEY is set")
else:
    print("UNSPLASH_ACCESS_KEY is not set in the environment.")

UNSPLASH_ACCESS_KEY is not set in the environment.


In [4]:
from googleapiclient.http import MediaIoBaseUpload
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from google.oauth2 import service_account
from pyunsplash import PyUnsplash
import os
from io import BytesIO
import requests
from PIL import Image, ImageEnhance

# Specify the path to your .env file (use an absolute path if needed)
dotenv_path = './key.txt'

# Load environment variables from the .env file
try:
    with open(dotenv_path, 'r') as file:
        for line in file:
            if line.strip() and not line.startswith('#'):
                key, value = line.strip().split('=', 1)
                os.environ[key] = value
except FileNotFoundError:
    print(f"Error: The .env file '{dotenv_path}' was not found.")

# Access your environment variables
UNSPLASH_ACCESS_KEY = os.environ.get("UNSPLASH_ACCESS_KEY")

if UNSPLASH_ACCESS_KEY:
    print("UNSPLASH_ACCESS_KEY is set in the environment.")
else:
    print("UNSPLASH_ACCESS_KEY is not set in the environment.")

UNSPLASH_ACCESS_KEY is set in the environment.


In [5]:
# instantiate PyUnsplash object
pu = PyUnsplash(api_key=UNSPLASH_ACCESS_KEY)

In [6]:
def search_and_download_from_unsplash(keys, n, folder_id):
    pics_list = []
    for _ in range(n):
        photos = pu.photos(type_='random', count=1, featured=True, query=keys)
        [photo] = photos.entries
        data_dict = {
            "id": photo.id,
            "url": photo.link_download
        }
        pics_list.append(data_dict)

    # Transform pics to grayscale and upload to Google Drive
    pics_gray = []
    credentials_file = 'cle.json'
    credentials = service_account.Credentials.from_service_account_file(
        credentials_file, scopes=['https://www.googleapis.com/auth/drive'])

    drive_service = build('drive', 'v3', credentials=credentials)

    for i, pic_data in enumerate(pics_list):
        response = requests.get(pic_data["url"], allow_redirects=True)
        if response.status_code == 200:
            image_data = BytesIO(response.content)
            image = Image.open(image_data)
            w, h = image.size
            if w > 2000:
                ratio = 1/3
                new = image.resize((round(w*ratio), round(h*ratio)))
            else:
                new = image
            gray_image = new.convert('L')
            pics_gray.append(gray_image)
        else:
            print("Failed to fetch the image.")

    for i, gray_image in enumerate(pics_gray):
        in_memory_file = BytesIO()
        gray_image.save(in_memory_file, format='PNG')

        file_metadata = {
            'name': f'image_{i}.png',
            'parents': [folder_id]
        }

        media = MediaIoBaseUpload(in_memory_file, mimetype='image/png')

        file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
        ).execute()

        print(f'Uploaded file ID for image "{keys}" {i}: {file["id"]}')

In [7]:
def upload_image_from_url(url, folder_id):
    try:
        # Download the image from the provided URL
        response = requests.get(url, allow_redirects=True)
        response.raise_for_status()  # Raise an error if the request was not successful

        # Check if the response contains image data
        if 'image' not in response.headers['Content-Type']:
            print("The provided URL does not point to an image.")
            return

        # Read the image data into a BytesIO object
        image_data = BytesIO(response.content)
        # Use PIL to open and process the image
        image = Image.open(image_data)
        # RESIZING (if needed)
        w, h = image.size
        if w > 2000:
            ratio = 1/3
            new = image.resize((round(w*ratio), round(h*ratio)))
        else:
            new = image
        # Convert the image to grayscale
        gray_image = new.convert('L')

        # Create an in-memory file from the grayscale image
        in_memory_file = BytesIO()
        # You can change the format as needed
        gray_image.save(in_memory_file, format='PNG')

        # Specify file metadata (customize the name and folder ID)
        file_metadata = {
            'name': 'uploaded_image.png',  # Customize the file name
            'parents': [folder_id]
        }

        credentials_file = 'cle.json'
        credentials = service_account.Credentials.from_service_account_file(
            credentials_file, scopes=['https://www.googleapis.com/auth/drive'])

        drive_service = build('drive', 'v3', credentials=credentials)

        # Create a media upload object from the in-memory file
        media = MediaIoBaseUpload(in_memory_file, mimetype='image/png')

        # Upload the file to Google Drive
        file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id'
        ).execute()

        # Print the file ID
        print(f'Uploaded file ID for the uploaded image: {file["id"]}')
    except requests.exceptions.RequestException as e:
        print(f"Error while fetching the image from the provided URL: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [8]:
def get_pics_by_demand():
    user_choice = input(
        "Enter 'U' to upload an image from a URL or 'S' to search and download it from Unsplash: ").lower()

    if user_choice == 'u':
        url = input("Enter the URL of the image you want to upload: ")
        folder_id = input(
            "Enter the Google Drive folder ID where you want to upload the image: ")
        upload_image_from_url(url, folder_id)
    elif user_choice == 's':
        keys = input("Enter your search keywords: ")
        n = int(input("Enter the number of photos you want to download: "))
        folder_id = input(
            "Enter the Google Drive folder ID where you want to upload the downloaded images: ")
        search_and_download_from_unsplash(keys, n, folder_id)
    else:
        print("Invalid choice. Please enter 'U' or 'S'.")

In [9]:
get_pics_by_demand()

Enter 'U' to upload an image from a URL or 'S' to search and download it from Unsplash: s
Enter your search keywords: free palestine
Enter the number of photos you want to download: 2
Enter the Google Drive folder ID where you want to upload the downloaded images: 1zj1ugP0gV2J5yWSQSsJgmD71zgoRS4IY
Uploaded file ID for image "free palestine" 0: 1TEUxlygAYIrHnnRG83qXFPPNsyRCJYcV
Uploaded file ID for image "free palestine" 1: 18_IGkHquFdA_eXRq2voeC1X1ohmKv510
